In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import random

In [2]:
import pandas as pd


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
driver = webdriver.Chrome(chromedriver)

In [5]:
rows=[]

In [6]:
#url1 = 'https://shop.rebag.com/collections/all-bags?page=1&_=pf&pf_st_availability_hidden=true&pf_t_categories=bc-filter-Bags'

In [7]:
how_many_pages = 4
for i in range(1, 11, 1):
    url_page = 'https://shop.rebag.com/collections/all-bags?page=' + str(i) + '&_=pf&pf_st_availability_hidden=true&pf_t_categories=bc-filter-Bags'
    driver.get(url_page)
    soup1 = BeautifulSoup(driver.page_source, 'html.parser')
    rows_one_page = soup1.find_all(class_='product-caption') #.find('a')
    rows.append(rows_one_page)
    time.sleep(.5+2*random.random())


In [8]:
links_bags = []
for row in rows:
    for r in row:
        links_bags.append(r.find('a')['href'])

In [9]:
len(links_bags)

480

In [10]:
links_bags

['/collections/all-bags/products/handbags-chanel-boy-flap-bag-quilted-caviar-old-medium5695319',
 '/collections/all-bags/products/handbags-chanel-coco-top-handle-bag-quilted-caviar-medium678761',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-leather-medium61020346',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-medium627171',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-mini685261',
 '/collections/all-bags/products/handbags-prada-promenade-bag-saffiano-leather-mini5709211',
 '/collections/all-bags/products/handbags-fendi-baguette-nm-bag-zucca-lycra-medium65203497',
 '/collections/all-bags/products/handbags-balenciaga-city-rubber-studs-bag-leather-mini5386756',
 '/collections/all-bags/products/handbags-louis-vuitton-multi-pochette-accessoires-monogram-canvas-619491',
 '/collections/all-bags/products/handbags-hermes-kelly-handbag-yellow-clemence-with-gold-hardware-32615781',
 '/co

In [11]:
def money_to_int(moneystring):
    if moneystring == 0:
        return None
    moneystring = moneystring.replace('$', '').replace(',', '').replace('USD', '').replace('Est.', '').replace('Retail', '')
    return int(moneystring)

In [12]:
def size_to_float(sizestring):
#Adapt volume of bag to float format

    if sizestring == 0:
        return None
    volume = 1
    sizestring = sizestring.replace('W', '').replace('H', '').replace('x', '')
    size_list = sizestring.split('"')
    if len(size_list) < 3:
        return None
    for i,el in enumerate(size_list):
        if i < 3:
            el = float(el.strip())
            volume = volume * el
            
    return volume


In [13]:
def get_details_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=field_name)
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [14]:
def get_bags_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://shop.rebag.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    headers = ['Title', 'Brand', 'Price', 'Condition', 'Retail_Price', 'Volume', 'Exterior Color',
               'Exterior Material', 'Interior Color', 'Interior Material', 'Accessories']
    
    
    #title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    title = title.rsplit(' ', 1)[0]
    print(title)
    
    if soup.find(class_='pdp__designer'):
        brand = soup.find(class_='pdp__designer').text
    else:
        brand = None
        
    if soup.find(itemprop='price'):
        raw_price = soup.find(itemprop='price').text 
        price = money_to_int(raw_price)
        #price = raw_price
    else:
        price=None
    #print(price)
    
    if soup.find(class_='pdp__condition-item--selected'):
        condition = soup.find(class_='pdp__condition-item--selected').text
    else:
        condition=None
    
    #!!! Change for good form!
    if soup.find(class_='pdp__retail-price'):
        raw_retail_price = soup.find(class_='pdp__retail-price').text
        retail_price = money_to_int(raw_retail_price)
        #retail_price = raw_retail_price
    else:
        retail_price=None
        
    ex_color = get_details_value(soup,'Exterior Color')
    #print(ex_color)
    
    ex_material = get_details_value(soup,'Exterior Material')
    #print(ex_material)
    
    int_color = get_details_value(soup,'Interior Color')
    
    int_material = get_details_value(soup,'Interior Material')
    
    # NEW!!!
    # Volume
    
    raw_volume = get_details_value(soup, 'SIZE AND FIT')
    if (raw_volume):
        volume = size_to_float(raw_volume)
    else:
        volume = None
    
    
    #Accessories
    accessories=[]
    for el_class in soup.find_all(class_="pdp__group-title"):
        if el_class.text == 'Accessories':
            for element in el_class.findNext().find_all('div'):
                accessories.append(element.text)
                accessories
    #
    
    bags_dict = dict(zip(headers, [title,
                                brand,
                                price,
                                condition,
                                retail_price,
                                volume,
                                ex_color,
                                ex_material,
                                int_color, 
                                int_material,
                                accessories]))

    return bags_dict
#, price

In [15]:
links_bags[0:10]

['/collections/all-bags/products/handbags-chanel-boy-flap-bag-quilted-caviar-old-medium5695319',
 '/collections/all-bags/products/handbags-chanel-coco-top-handle-bag-quilted-caviar-medium678761',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-leather-medium61020346',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-medium627171',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-mini685261',
 '/collections/all-bags/products/handbags-prada-promenade-bag-saffiano-leather-mini5709211',
 '/collections/all-bags/products/handbags-fendi-baguette-nm-bag-zucca-lycra-medium65203497',
 '/collections/all-bags/products/handbags-balenciaga-city-rubber-studs-bag-leather-mini5386756',
 '/collections/all-bags/products/handbags-louis-vuitton-multi-pochette-accessoires-monogram-canvas-619491',
 '/collections/all-bags/products/handbags-hermes-kelly-handbag-yellow-clemence-with-gold-hardware-32615781']

List of all bags, let's try open all pages from this page

In [16]:
bags_page_info_list = []

In [17]:
bags_page_info_list = []
for link in links_bags:
    bags_page_info_list.append(get_bags_dict(link))
    time.sleep(5+2*random.random())

Chanel Boy Flap Bag Quilted Caviar Old Medium
Chanel Coco Top Handle Bag Quilted Caviar Medium
Christian Dior Saddle Handbag Leather Medium
Christian Dior Saddle Handbag Oblique Canvas Medium
Christian Dior Saddle Handbag Oblique Canvas Mini
Prada Promenade Bag Saffiano Leather Mini
Fendi Baguette NM Bag Zucca Lycra Medium
Balenciaga City Rubber Studs Bag Leather Mini
Louis Vuitton Multi Pochette Accessoires Monogram Canvas
Hermes Kelly Handbag Natural Sable Clemence with Gold Hardware 32
Louis Vuitton Pochette Accessoires Damier Mini
Chanel Flat Handle Boy Flap Bag Quilted Lambskin Old Medium
Christian Dior Ultra Matte Lady Dior Bag Cannage Quilt Calfskin Medium
Louis Vuitton Multi Pochette Accessoires Monogram Canvas
Christian Dior Diorama Wallet on Chain Satin
Hermes Vintage Cadena Top Handle Bag Box Calf
Chanel Coco Envelope Flap Bag Chevron Leather Medium
Saint Laurent Classic Monogram Tassel Chain Wallet Leather
Saint Laurent Rive Gauche Shopper Tote Canvas Large
Chanel Square Cl

Chanel Vintage CC Chain Tote Caviar Small
Louis Vuitton Boite Chapeau Souple Bag Monogram Canvas MM
Saint Laurent Classic Monogram Wallet on Chain Matelasse Chevron Leather Medium
Prada Bowling Convertible Handle Bag Leather Small
Louis Vuitton Moon Alma Bag Monogram Canvas
Hermes Birkin Handbag Rose Tyrien Epsom with Gold Hardware 35
Chanel Coco Top Handle Bag Chevron Calfskin Large
Louis Vuitton Alma Handbag Monogram Vernis BB
Saint Laurent Sunset Crossbody Bag Crocodile Embossed Leather Medium
Chanel Vintage Timeless Zip Around Vanity Case Patent Small
Chanel Coco Top Handle Bag Chevron Calfskin Large
Chanel Vintage Diamond Stitch Drawstring Bucket Bag Quilted Lambskin
Salvatore Ferragamo Amy Tote Pebbled Leather Mini
Louis Vuitton Saint Jacques Handbag Epi Leather PM
Chanel Boy Flap Bag Quilted Lambskin New Medium
Fendi Wave Double Baguette Leather Micro
Hermes Evelyne Crossbody Gen III Epsom PM
Chanel Vintage Trapezoid CC Flap Bag Leather Small
Prada
Chanel Mosaic Flap Bag Embelli

Saint Laurent Uptown Tote Canvas and Leather Medium
Louis Vuitton Artsy Handbag Monogram Empreinte Leather MM
Christian Dior My Lady Dior Bag Cannage Quilted Leather
Chanel Classic Double Flap Quilt Print Patent Medim
Chloe Faye Day Bag Leather Small
Louis Vuitton Neverfull Tote Damier MM
Chanel Business Affinity Camera Case Bag Quilted Caviar Small
Chanel CC Chain Flap Fur with Faux Pearl Detail Medium
Chanel Vintage Timeless Boston Bag Caviar Medium
Chanel Wallet on Chain Quilted Lambskin
Chanel CC Chain Drawstring Bucket Bag Quilted Lambskin Small
Saint Laurent Sunset Top Handle Chain Wallet Leather
Saint Laurent Sunset Wallet on Strap Leather
Hermes Kelly Handbag Bleu Indigo Swift with Palladium Hardware 25
Chanel Embossed Logo Bowler Bag Quilted Leather Medium
Gucci Ophidia Handle Shoulder Bag GG Coated Canvas Mini
Chloe Hudson Handbag Studded Leather Mini
Saint Laurent Le 61 Camera Bag Leather
Prada Elektra Shoulder Bag Studded Leather Small
Christian Dior Roller Messenger Bag Ob

Chanel Citizen Flap Bag Quilted Calfskin Mini
Hermes Kelly Handbag Etrusque Buffalo Skipper with Gold Hardware 35
Chanel Vintage Front Pocket Camera Bag Quilted Leather Small


In [18]:
bags_page_info_list

[{'Title': 'Chanel Boy Flap Bag Quilted Caviar Old Medium',
  'Brand': 'Chanel',
  'Price': 4815,
  'Condition': 'Pristine',
  'Retail_Price': None,
  'Volume': 165.0,
  'Exterior Color': 'Pink',
  'Exterior Material': 'Leather',
  'Interior Color': 'Pink',
  'Interior Material': 'Fabric',
  'Accessories': ['Box', 'Dust bag', 'Authenticity card']},
 {'Title': 'Chanel Coco Top Handle Bag Quilted Caviar Medium',
  'Brand': 'Chanel',
  'Price': 4060,
  'Condition': 'Very Good',
  'Retail_Price': 4300,
  'Volume': 540.0,
  'Exterior Color': 'Neutral',
  'Exterior Material': 'Leather',
  'Interior Color': 'Neutral',
  'Interior Material': 'Fabric',
  'Accessories': ['Dust bag', 'Detachable Strap']},
 {'Title': 'Christian Dior Saddle Handbag Leather Medium',
  'Brand': 'Christian Dior',
  'Price': 3220,
  'Condition': 'Great',
  'Retail_Price': None,
  'Volume': 270.0,
  'Exterior Color': 'Black',
  'Exterior Material': 'Leather',
  'Interior Color': 'Black',
  'Interior Material': 'Suede',


In [19]:
bags_page_info = pd.DataFrame(bags_page_info_list)  #convert list of dict to df
bags_page_info.set_index('Title', inplace=True)

bags_page_info

,Brand,Price,Condition,Retail_Price,Volume,Exterior Color,Exterior Material,Interior Color,Interior Material,Accessories
Title,,,,,,,,,,
Chanel Boy Flap Bag Quilted Caviar Old Medium,Chanel,4815.0,Pristine,NaN,165.0,Pink,Leather,Pink,Fabric,"[Box, Dust bag, Authenticity card]"
Chanel Coco Top Handle Bag Quilted Caviar Medium,Chanel,4060.0,Very Good,4300.0,540.0,Neutral,Leather,Neutral,Fabric,"[Dust bag, Detachable Strap]"
Christian Dior Saddle Handbag Leather Medium,Christian Dior,3220.0,Great,NaN,270.0,Black,Leather,Black,Suede,[No accessories]
Christian Dior Saddle Handbag Oblique Canvas Medium,None,NaN,None,NaN,NaN,None,None,None,None,[]
Christian Dior Saddle Handbag Oblique Canvas Mini,Christian Dior,3390.0,Excellent,NaN,96.0,Blue,Canvas (Plain),Neutral,Fabric,[No accessories]
...,...,...,...,...,...,...,...,...,...,...
Gucci Dionysus Chain Wallet GG Coated Canvas Small,None,NaN,None,NaN,NaN,None,None,None,None,[]
Hermes Birkin Handbag Vermillon Swift with Gold Hardware 25,Hermes,14565.0,Very Good,NaN,400.0,Red,Leather,Red,Leather,"[Clochette, Keys, Lock, Dust bag, Box]"
Chanel Citizen Flap Bag Quilted Calfskin Mini,Chanel,2575.0,Very Good,NaN,78.0,Black,Leather,Black,Satin,[Authenticity card]


In [20]:
bags_page_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, Chanel Boy Flap Bag Quilted Caviar Old Medium to Chanel Vintage Front Pocket Camera Bag Quilted Leather Small
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              291 non-null    object 
 1   Price              291 non-null    float64
 2   Condition          290 non-null    object 
 3   Retail_Price       156 non-null    float64
 4   Volume             291 non-null    float64
 5   Exterior Color     291 non-null    object 
 6   Exterior Material  291 non-null    object 
 7   Interior Color     289 non-null    object 
 8   Interior Material  290 non-null    object 
 9   Accessories        480 non-null    object 
dtypes: float64(3), object(7)
memory usage: 41.2+ KB


In [23]:
bags_page_info.to_csv('bags_page_info.csv')

In [22]:
bags_page_info

,Brand,Price,Condition,Retail_Price,Volume,Exterior Color,Exterior Material,Interior Color,Interior Material,Accessories
Title,,,,,,,,,,
Chanel Boy Flap Bag Quilted Caviar Old Medium,Chanel,4815.0,Pristine,NaN,165.0,Pink,Leather,Pink,Fabric,"[Box, Dust bag, Authenticity card]"
Chanel Coco Top Handle Bag Quilted Caviar Medium,Chanel,4060.0,Very Good,4300.0,540.0,Neutral,Leather,Neutral,Fabric,"[Dust bag, Detachable Strap]"
Christian Dior Saddle Handbag Leather Medium,Christian Dior,3220.0,Great,NaN,270.0,Black,Leather,Black,Suede,[No accessories]
Christian Dior Saddle Handbag Oblique Canvas Medium,None,NaN,None,NaN,NaN,None,None,None,None,[]
Christian Dior Saddle Handbag Oblique Canvas Mini,Christian Dior,3390.0,Excellent,NaN,96.0,Blue,Canvas (Plain),Neutral,Fabric,[No accessories]
...,...,...,...,...,...,...,...,...,...,...
Gucci Dionysus Chain Wallet GG Coated Canvas Small,None,NaN,None,NaN,NaN,None,None,None,None,[]
Hermes Birkin Handbag Vermillon Swift with Gold Hardware 25,Hermes,14565.0,Very Good,NaN,400.0,Red,Leather,Red,Leather,"[Clochette, Keys, Lock, Dust bag, Box]"
Chanel Citizen Flap Bag Quilted Calfskin Mini,Chanel,2575.0,Very Good,NaN,78.0,Black,Leather,Black,Satin,[Authenticity card]
